In [ ]:
import numpy as np
import matplotlib.pyplot as plt 
from scipy.linalg import solve_banded

# We will try to solve for the function zeta using implict method and compare our answer to the one obtained through backward differenc
# Kriess Oliger Dissipation function has also been employed

def round_of_to_lowest_ten(x):
    return (x // 10)* 10 

def get_banded_form(matrix, bandwidth_up, bandwidth_down):
    n = matrix.shape[0]
    diagonals = np.zeros((bandwidth_up + bandwidth_down + 1, n))

    for i in range(-bandwidth_down, bandwidth_up + 1):
        for j in range(0, n - abs(i)):
            if i >= 0:
                diagonals[bandwidth_up - i, n - 1 -j] = np.diag(matrix, k = i)[n - abs(i) - 1 - j]
            else:
                diagonals[bandwidth_up - i, j] = np.diag(matrix, k = i)[j]
    return diagonals

def zeta_zero(p_given):
    m = 1
    Q = 0.9
    p = p_given
    A = 0.5
    b_couple = 200
    y_boundary =2* m 
    a = 2000           # Kreiss Oliger Dissipation coefficient
    h = 1 / 2**12       # Step size
    n = 2**12 - 1       # We only solve for the points which are not initially fixed

    # Definiing the implicit matrix
    M = np.zeros([n, n])

    # Defining the boundary parts of matrix 
    M[0, :5]         = [-25 + 12 * a * h * 35, 48 + 12 * a * h * (-104), -36 + 12 * a * h * 114, 16 + 12 * a * h * (-56), -3 + 12 * a * h * 11]
    M[1, :5]         = [-3 + 12 * a * h * 11, -10 + 12 * a * h * (-20), 18 + 12 * a * h * 6, -6 + 12 * a * h * 4, 1 + 12 * a * h * (-1)]
    M[n - 2, n - 4:] = [1 + 12 * a * h * (-1), -8 + 12 * a * h * 16, 0 + 12 * a * h * (-30), 8 + 12 * a * h * 16]
    M[n - 1, n - 4:] = [-1 + 12 * a * h * (-1), 6 + 12 * a * h * 4, -18 + 12 * a * h * 6, 10 + 12 * a * h * (-20)]
    
    # Bulk of the matrix
    for i in range(2, n - 2):       
        M[i][i - 2] = 1 + 12* a* h* (-1)
        M[i][i - 1] = -8 + 12* a* h* (16)
        M[i][i]     = 0 + 12* a* h* (-30) 
        M[i][i + 1] = 8 + 12* a* h* (16)
        M[i][i + 2] = -1 + 12* a* h* (-1)

    err = 10**(-3)
    Z = np.linspace(err, 1 - err, 2**(12))

    Z_mod = Z[:-1]      # Neglecting the final point
    b = np.zeros(n)
    
    for i in range(0, n):
            c1 = Q**2 / (m* Z_mod[i]**2)
            c2 = 4* A**2* m* Z_mod[i]**4 / (p**4 * (Z_mod[i] - 1)**6 )

            int1 = np.exp(- 4* Z_mod[i]**2 / (p**2* (Z_mod[i] - 1)**2 ))
            int2 = np.exp(- 2* Z_mod[i]**2 / (p**2* (Z_mod[i] - 1)**2 ))

            term1 = c1* np.exp( - A**4* b_couple* int1 ) 
            term2 = c2* int2
            term = term1 + term2    
            b[i] = term* 12* h

    
    # Fixing initial conditions
    b[n - 1] = b[n - 1] - ((3 + 11* 12*a*h)* y_boundary) 
    b[n - 2] = b[n - 2] - ((-1 - 1* 12*a*h)* y_boundary) 

    """
    M_inv = np.linalg.inv(M)
    Y = np.dot(M_inv,  b)
    """

    # Using banded technique
    banded_M = get_banded_form(M, 4, 3)
    Y = solve_banded((3, 4), banded_M, b)
    
    Y_mod = np.append(Y, y_boundary)
    R = m* Z/(1 - Z)

    cut_off = round_of_to_lowest_ten(np.argmin(abs(Y_mod - R)))

    print(f" The position of initial apparant horizon is r = {R[cut_off]} \n z = {Z[cut_off]} \n m = {cut_off}") 
    plt.plot(Z_mod[n//5:4*n//5], Y[n//5:4*n//5])
    plt.plot(Z_mod[n//5:4*n//5], R[n//5:4*n//5])
    plt.show()
    
    m = n - cut_off + 1
    s = np.sqrt(Y_mod[cut_off:])
    r = R[cut_off:]
    z = Z[cut_off:]

    return m, z, r, s

# zeta_zero(2)


In [ ]:
import numpy as np
from scipy.linalg import solve_banded

# Function to covert the matrix to banded form
def get_banded_form(matrix, bandwidth_up, bandwidth_down):
    
    n = matrix.shape[0]
    diagonals = np.zeros((bandwidth_up + bandwidth_down + 1, n))

    for i in range(-bandwidth_down, bandwidth_up + 1):
        for j in range(0, n - abs(i)):
            if i >= 0:
                diagonals[bandwidth_up - i, n - 1 -j] = np.diag(matrix, k = i)[n - abs(i) - 1 - j]
            else:
                diagonals[bandwidth_up - i, j] = np.diag(matrix, k = i)[j]
    return diagonals


def Updated_implicit_Alpha(array_size, step_size, d_dz, r, Y):
    
    n = array_size - 1
    h = step_size
    a = 2000             # Dissipation coefficient for alpha
    phi = Y[0]
    s = Y[1]
    pi = Y[2]

    # Defining the auxillary variable l = log(alpha) which we will solve for instead of alpha
    # boundary condition
    l_ini = 0
    
    # We will be using the implicit method to solve this equation
    # Inititalizing the Right hand side of the equation
    b =  - 12* h* r[:-1]* np.sqrt(r[:-1])* pi[:-1]* d_dz(phi[:-1]) / s[:-1]

    # As the boundary condition is zero we dont need too subtract with anything, therefore no need to construct b_modified

    # Defining the matrix
    M = np.zeros([n, n])

    # Defining the boundary parts of matrix 
    M[0, :5]         = [-25 + 12 * a * h * 35, 48 + 12 * a * h * (-104), -36 + 12 * a * h * 114, 16 + 12 * a * h * (-56), -3 + 12 * a * h * 11]
    M[1, :5]         = [-3 + 12 * a * h * 11, -10 + 12 * a * h * (-20), 18 + 12 * a * h * 6, -6 + 12 * a * h * 4, 1 + 12 * a * h * (-1)]
    M[n - 2, n - 4:] = [1 + 12 * a * h * (-1), -8 + 12 * a * h * 16, 0 + 12 * a * h * (-30), 8 + 12 * a * h * 16]
    M[n - 1, n - 4:] = [-1 + 12 * a * h * (-1), 6 + 12 * a * h * 4, -18 + 12 * a * h * 6, 10 + 12 * a * h * (-20)]
    
    # Bulk of the matrix
    for i in range(2, n - 2):       
        M[i][i - 2] = 1 + 12* a* h* (-1)
        M[i][i - 1] = -8 + 12* a* h* (16)
        M[i][i]     = 0 + 12* a* h* (-30) 
        M[i][i + 1] = 8 + 12* a* h* (16)
        M[i][i + 2] = -1 + 12* a* h* (-1)

    banded_M = get_banded_form(M, 4, 3)
    l_int  = solve_banded((3, 4), banded_M, b)

    # M_inv = np.linalg.inv(M)
    # l_int = np.dot(M_inv, b)

    # Adding back the final value
    l = np.append(l_int, 0)

    # Returning alpha = e^l
    return np.exp(l)

In [ ]:
def one_iter_RK4(t, Y, h, System):

    n_sys = len(System)
    m = len(Y[0])

    k1 = np.zeros([n_sys, m])
    k2 = np.zeros([n_sys, m])
    k3 = np.zeros([n_sys, m])
    k4 = np.zeros([n_sys, m])

    for j in range(0, n_sys):
        k1[j] = h* (System[j](t, Y))

    # alpha.append(Updated_implicit_Alpha(m, dz, d_dz, r, Y + k1/2))

    for j in range(0, n_sys):
        k2[j] = h* (System[j](t + h/2, Y + k1/2))
        
    # alpha.append(Updated_implicit_Alpha(m, dz, d_dz, r, Y + k2/2))

    for j in range(0, n_sys):
        k3[j] = h* (System[j](t + h/2, Y + k2/2))

    # alpha.append(Updated_implicit_Alpha(m, dz, d_dz, r, Y + k3))

    for j in range(0, n_sys):
        k4[j] = h* (System[j](t + h, Y + k3))

    Y_next = Y + (k1 + 2* k2 + 2* k3 + k4)/6
    t_next = t + h

    return t_next, Y_next

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from findiff import FinDiff

def evol(p_given):
    M = 1
    Q = 0.9
    b = 200
    p = p_given
    A = 0.5
    # D = 0.1       # Kreiss Oliger dissipation coefficient
    n = 2**(12)     # Defining the number of grid points

    # Calling the implicit s function to get our initial data. We will be working with s0 instead of s beacuse of the decay issues

    m, z, r, s0 = zeta_zero(p)

    # Defining Alpha at t = 0
    alpha0 = np.ones(m)

    # Defining the alpha list as its not part of the evolving ODEs and just constraint at every point in time.
    # Hence has be manually updated, which the list definition makes it easier to do so
    alpha = [alpha0]    

    # Defining Pi at t = 0
    pi0 = np.zeros(m) 

    # Defining Phi in the required range at t = 0
    phi0 = A* np.exp(- z**2 / (p**2 * (1 - z)**2 ))

    # Defining the derivative operator from findiff
    dz = z[1] - z[0]
    d_dz = FinDiff(0, dz, 1, acc = 4)


    # Defining the time interval
    t_ini = 0
    t_final = 100
    dt_ini = 0.001
    delta = 10**(-2)
    n_sys = 3 # No of simultaneous system of equations we are trying to evlove

    # Defining the RHS of the differential equations 

    def phi(t, Y):
        Cap_phi = d_dz(Y[0]) / d_dz(r)
        return alpha[-1]* (Y[2] + Cap_phi* Y[1] / np.sqrt(r))

    def s(t, Y):
        Cap_phi = d_dz(Y[0]) / d_dz(r)
        return (r**2 * alpha[-1] / Y[1])* (Y[2] + Cap_phi* Y[1]/ np.sqrt(r))* (Y[2]* Y[1]/ np.sqrt(r) + Cap_phi)
        
    def Pi(t, Y):
        Cap_phi = d_dz(Y[0]) / d_dz(r)
        term1 = d_dz( (Y[2]* Y[1]/np.sqrt(r)  + Cap_phi)* alpha[-1]* r**2 ) / d_dz(r)
        term2 = alpha[-1]* Q**2 * 4* b* np.exp(-b* Y[0]**4 )* Y[0]**3 / (2* r**4)
        return term1/r**2 - term2

    # Defining the system of ODE to be easier to iterate over
    System = [phi, s, Pi]               

    # Defining the time - grid points
    T = [t_ini]

    # Setting the initial Conditions at t = 0 in the funcs array 
    Initial_conditions = np.array([phi0, s0, pi0])
    Funcs = [Initial_conditions]
    
    # Defining an array to store step sizes
    H = [dt_ini]

    while T[-1] <= t_final:

        temp_T, temp_Y = one_iter_RK4(T[-1], Funcs[-1], H[-1],  System)

        temp_h = one_iter_RK4(temp_T, temp_Y, H[-1], System)[1]
        temp_2h = one_iter_RK4(temp_T, temp_Y, 2*H[-1], System)[1]

        # Considering only first 500 points of phi for the error estimates cause those are the important values

        # Postion of apparant horizon
        ones = np.ones(m)
        pos_ap = np.argmin(abs(temp_h[1]/np.sqrt(r) - ones))
        diff = abs(temp_2h[0][pos_ap] - temp_h[0][pos_ap])

        # Here I have given equal importance to every point that we are calculating
        rho = 30* H[-1]* delta / diff
        # print(f"rho = {rho}")

        if rho > 16:
            T.append(T[-1] + H[-1])
            T.append(T[-1] + 2* H[-1])
            Funcs.append(temp_h)
            Funcs.append(temp_2h)
            H.append(2* H[-1])
            alpha.append(Updated_implicit_Alpha(m, dz, d_dz, r, Funcs[-1]))
        
        elif rho > 1:
            T.append(T[-1] + H[-1])
            T.append(T[-1] + 2* H[-1])
            Funcs.append(temp_h)
            Funcs.append(temp_2h)
            H.append(H[-1]* rho**(1/4))
            alpha.append(Updated_implicit_Alpha(m, dz, d_dz, r, Funcs[-1]))

        else:
            print("case3")
            H[-1] = H[-1]* rho**(1/4)

        print(H[-1], T[-1], len(T))

    # Now we have the values of every metric/ matter function at all points for every time instances

    N = len(T)
    apparant_horizon_phi = np.zeros(N)
    apparant_horizon_radius = np.zeros(N)

    ones = np.ones(m)
    for i in range(0, N):
        index = np.argmin(abs(Funcs[i][1]/np.sqrt(r) - ones))
        apparant_horizon_phi[i]    = Funcs[i][0][index]
        apparant_horizon_radius[i] = r[index]

    return Funcs, apparant_horizon_phi, apparant_horizon_radius, T



In [ ]:
import matplotlib.pyplot as plt

p_star = 2.116895853824
funcs1, p1, r1, t1 =  evol(p_star + 10**(-12))
# funcs2, p2, r2, t2 =  evol(1.9)

plt.plot(t1, p1)
# plt.plot(t2, p2)
plt.show()

plt.plot(t1, r1)
# plt.plot(t2, r2)
plt.show()

In [ ]:
N = 2**(12)
Z = np.linspace(0, 1, N)
z = Z[2330:]
plt.plot(z, funcs1[0, 0, :], label = '0')
plt.plot(z, funcs1[10, 0, :],label = '100')
plt.plot(z, funcs1[50, 0, :],label = '500')
plt.plot(z, funcs1[100, 0, :],label = '1000')
plt.plot(z, funcs1[500, 0, :],label = '5000')
plt.plot(z, funcs1[999, 0, :],label = '10000')
plt.legend()
plt.show()

In [ ]:
dt = (1 - 0)/ 1000
d_dt = FinDiff(0, dt, 1, acc = 2)
dp1_dt = d_dt(p1)
print(dp1_dt.shape)
plt.plot(t1, dp1_dt, 'o')
plt.show()